# Datacleaning Pandas

In [355]:
import pandas as pd
import numpy as np

#DataFrame
df=pd.DataFrame({'From_To':['LoNDon_paris','MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis','Brussels_londOn'],'FlightNumber':[10045,np.nan,10065,np.nan,10085],'RecentDelays':[[23,47],[],[24,43,87],[13],[67,32]],'Airline':['KLM(!)','<Air France>(12)','(British Airways. )','12.Air France','''Swiss Air''']})

In [356]:

df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France>(12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12.Air France
4,Brussels_londOn,10085.0,"[67, 32]",Swiss Air


In [357]:
# NULL values in the dataframe
df.isna().sum()

From_To         0
FlightNumber    2
RecentDelays    0
Airline         0
dtype: int64

# 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put inplace. Fill in these missing numbers and make the column an integer column(instead of a float column).

In [358]:
#Filling the NaN values with the previous values
df.fillna(method='ffill')

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10045.0,[],<Air France>(12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10065.0,[13],12.Air France
4,Brussels_londOn,10085.0,"[67, 32]",Swiss Air


In [359]:
# notnull function returns boolean values according to null and not null values
df.FlightNumber.notnull()

0     True
1    False
2     True
3    False
4     True
Name: FlightNumber, dtype: bool

In [360]:
#True=1 False=0 Cummulative sum of the return values of notnull function is calculated
df.FlightNumber.notnull().cumsum()


0    1
1    1
2    2
3    2
4    3
Name: FlightNumber, dtype: int64

In [361]:
#Numbering each item in each group from 0 to the length of that group - 1.
df.groupby(df.FlightNumber.notnull().cumsum()).cumcount()

0    0
1    1
2    0
3    1
4    0
dtype: int64

In [362]:

df['FlightNumber']=df.FlightNumber.ffill()+df.groupby(df.FlightNumber.notnull().cumsum()).cumcount()*10

In [363]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France>(12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12.Air France
4,Brussels_londOn,10085.0,"[67, 32]",Swiss Air


In [365]:
df.dtypes # the type of flightnumber column is float

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

In [366]:
df['FlightNumber'] = df['FlightNumber'].astype(int) # changing the type of column FlightNumber from float to int
  

In [354]:
df.dtypes

From_To         object
FlightNumber     int64
RecentDelays    object
Airline         object
dtype: object

# 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [277]:
df.From_To.str.split("_",expand=True)

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [278]:
#split(delimiter,n) n is the limit of splits if there are multiple delimiters
df_new=pd.DataFrame(df.From_To.str.split('_',1).tolist(),
                         columns = ['From','To'])

In [279]:
df_new# temporary dataframe

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


# 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)


In [280]:
df['From_To'] = df['From_To'].str.title()

In [281]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,London_Paris,10045.0,"[23, 47]",KLM(!)
1,Madrid_Milan,10055.0,[],<Air France>(12)
2,London_Stockholm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_Paris,10075.0,[13],12.Air France
4,Brussels_London,10085.0,"[67, 32]",Swiss Air


In [282]:
df.drop('From_To',axis=1,inplace=True)

In [283]:
df

,FlightNumber,RecentDelays,Airline
0,10045.0,"[23, 47]",KLM(!)
1,10055.0,[],<Air France>(12)
2,10065.0,"[24, 43, 87]",(British Airways. )
3,10075.0,[13],12.Air France
4,10085.0,"[67, 32]",Swiss Air


# 4. Delete the From_To column from df and attach the temporary DataFramefrom the previous questions.


In [284]:
df_new_final=df_new.join(df)

In [285]:
df_new_final

,From,To,FlightNumber,RecentDelays,Airline
0,LoNDon,paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid,miLAN,10055.0,[],<Air France>(12)
2,londON,StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest,PaRis,10075.0,[13],12.Air France
4,Brussels,londOn,10085.0,"[67, 32]",Swiss Air


## 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [296]:
delays = pd.DataFrame(df_new_final.RecentDelays.tolist(), index= df.index,columns=['delay_1','delay_2','delay_3'])

In [297]:
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [302]:
df=df.join(delays)

In [307]:

df.drop('RecentDelays',axis=1,inplace=True)

In [308]:
df

,FlightNumber,Airline,delay_1,delay_2,delay_3
0,10045.0,KLM(!),23.0,47.0,NaN
1,10055.0,<Air France>(12),NaN,NaN,NaN
2,10065.0,(British Airways. ),24.0,43.0,87.0
3,10075.0,12.Air France,13.0,NaN,NaN
4,10085.0,Swiss Air,67.0,32.0,NaN
